# Port Statistics

This notebook develops and explores the various port statistics used in the [Port Performance Project](https://github.com/epistemetrica/Port-Performance-Project). See the README.md file in the main directory for more info.

The primary data set comes from a combination of AIS vessel data and port data, processed in the Port Geodata notebook.

Statistics and final dataframes developed here are used in the Port Performance Dashboard.



In [194]:
#prelims
import polars as pl
import polars.selectors as cs
import pandas as pd
import geopandas as gpd
import time
import plotly.express as px
import matplotlib.pyplot as plt
import contextily as cx
import numpy as np
import glob
import folium
from folium.plugins import HeatMap

#enable string cache for polars categoricals
pl.enable_string_cache()
#display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pl.Config(tbl_rows=100);

## Load Data from geodata_prep notebook

In [195]:
#load data from parquet
main_lf = pl.scan_parquet('port data/dashboard/main.parquet')
#inspect
display(main_lf.describe())
main_lf.limit(5).collect()

statistic,mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,which_port_waters,port_waters1,port_waters2,port_waters3,status_duration,dock_id,dock_name,facility_type,port_name,vessel_lat,vessel_lon,dock_lat,dock_lon,port_lat,port_lon,call_id
str,str,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,str
"""count""","""1479131""","""1479131""",1.479131e6,1.476266e6,1.459126e6,1.47913e6,"""1479131""",1.479131e6,1.479121e6,1.479131e6,1.374029e6,1.369554e6,1.181491e6,"""1479131""","""1479131""","""1479131""","""1479131""","""1475498""","""1479131""","""1479131""","""1474646""","""1479131""",1.479131e6,1.479131e6,1.479131e6,1.479131e6,1.479131e6,1.479131e6,"""1479131"""
"""null_count""","""0""","""0""",0.0,2865.0,20005.0,1.0,"""0""",0.0,10.0,0.0,105102.0,109577.0,297640.0,"""0""","""0""","""0""","""0""","""3633""","""0""","""0""","""4485""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,"""0"""
"""mean""",null,"""2021-06-20 13:08:44.930534""",2.069134,183.956445,186.137288,1.990291,null,73.030351,1.0292e7,208.684358,31.890638,10.748653,73.64155,null,null,null,null,"""12:57:35.342323""",null,null,null,null,32.421945,-96.524135,32.301671,-96.7059,32.300501,-96.706908,null
"""std""",null,null,4.787626,91.86207,103.02194,2.446991,null,4.782461,2.8644e7,58.168777,7.610614,2.91223,8.057182,null,null,null,null,null,null,null,null,null,6.433952,19.987501,6.335007,19.913109,6.334674,19.912602,null
"""min""","""205042000""","""2018-01-01 00:35:19""",0.0,0.0,0.0,0.0,null,70.0,0.0,101.0,0.0,-12.8,0.0,"""Albany Port District, NY""","""Albany Port District, NY""","""Guaynabo, PR""","""Not in port waters""","""0:05:01""","""00XE""","""ADM Corpus Christi Grain Eleva…","""Anchorage""","""Albany Port District, NY""",12.37642,-170.62387,17.936081,-166.53444,17.938939,-166.549916,"""205042000_Port of Long Beach, …"
"""25%""",null,"""2019-10-09 13:20:59""",0.0,122.8,93.0,0.0,null,70.0,9.314624e6,179.0,28.0,8.7,70.0,null,null,null,null,"""0:57:02""",null,null,null,null,29.30762,-118.21407,29.308056,-118.21888,29.31049,-118.2095,null
"""50%""",null,"""2021-08-07 16:13:13""",0.1,184.0,182.0,0.0,null,70.0,9.484326e6,189.0,32.0,10.6,70.0,null,null,null,null,"""1:58:41""",null,null,null,null,30.66838,-93.95915,30.668867,-93.968517,30.69123,-93.96069,null
"""75%""",null,"""2023-01-31 22:32:19""",0.2,252.3,271.0,5.0,null,80.0,9.683427e6,229.0,32.0,12.8,80.0,null,null,null,null,"""4:18:00""",null,null,null,null,36.8752,-80.12159,34.719167,-80.122222,34.71693,-80.117801,null
"""max""","""725019920""","""2024-09-30 23:22:34""",102.3,359.9,359.0,15.0,null,89.0,9.901843e8,667.0,61.0,25.5,191.0,"""Yabucoa, PR""","""Yabucoa, PR""","""Yabucoa, PR""","""Yabucoa, PR""","""2191 days, 16:36:18""","""1JHK""","""YUSEN TERMINALS BERTHS 212-221""","""Tie Off""","""Wilmington, NC""",61.24418,144.6713,61.24306,-66.086926,61.23778,-66.096678,"""725019920_Port Everglades, FL_…"


mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,which_port_waters,port_waters1,port_waters2,port_waters3,status_duration,dock_id,dock_name,facility_type,port_name,vessel_lat,vessel_lon,dock_lat,dock_lon,port_lat,port_lon,call_id
str,datetime[μs],f64,f64,f64,f64,cat,f64,f64,f64,f64,f64,f64,str,str,str,str,duration[μs],str,str,str,str,f64,f64,f64,f64,f64,f64,str
"""205042000""",2022-12-07 23:20:37,9.1,358.1,0.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",2h 34m 41s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.36623,-118.04241,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-08 01:55:18,0.3,83.8,328.0,1.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",6d 18h 3m 47s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.62433,-118.0504,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-14 19:59:05,0.9,232.6,228.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",4h 35m 49s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.62758,-118.05079,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-15 00:34:54,0.0,340.4,340.0,5.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",1d 8h 25m 12s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.75624,-118.21814,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-16 09:00:06,0.2,81.3,339.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",1h 2m 19s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.75624,-118.21808,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"


## Generate Stats

In [196]:
#get stats for each call
calls_df = (
    main_lf
    #ensure sorting by vessel and time
    .sort(['mmsi', 'time'])
    #drop calls with missing status duration (indicating AIS went offline while in port waters)
    .filter(pl.col('status_duration').is_not_null().over('call_id'))
    #grouby by call id
    .group_by('call_id')
    .agg(
        #port name
        port_name = pl.first('port_name'),
        #dock name
        dock_name = pl.first('dock_name'),
        #dock_id
        dock_id = pl.first('dock_id'),
        #vessel mmsi
        mmsi = pl.first('mmsi'),
        #vessel imo
        imo = pl.first('imo'),
        #vessel size
        vessel_size = pl.first('length'),
        #time entering port waters
        time_port_entry = pl.col('time').min(),
        #time of arrival at dock
        time_arrival = (
            pl.when(pl.col('status')==5)
            .then(pl.col('time'))
            .otherwise(pl.lit(None))
        ).min(),
        #time of departure from dock
        time_departure = (
            pl.when(pl.col('status')==5)
            .then(pl.col('time')+pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).max(),
        #time port exit
        time_port_exit = (pl.col('time') + pl.col('status_duration')).max(),
        #hrs a berth
        hrs_at_berth = (
            ((pl.col('status')==5)*
            (pl.col('status_duration').dt.total_minutes()/60)
            )
        ).sum(),
        #hrs at anchor
        hrs_at_anchor = (
            pl.when(pl.col('status')==1)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
            .sum().dt.total_minutes()/60
        )
    )
    #drop calls with missing arrival or departure time
    .filter(pl.col('time_arrival').is_not_null() & 
            pl.col('time_departure').is_not_null())
    #drop calls with missing port entry or exit time
    .filter(pl.col('time_port_entry').is_not_null() & 
            pl.col('time_port_exit').is_not_null())
    #compute additional stats
    .with_columns(
        #time from port entry to docking in hrs
        hrs_to_dock = (
            (pl.col('time_arrival') - pl.col('time_port_entry'))
            .dt.total_minutes()/60
        ),
        #time in port waters after leaving dock
        hrs_in_port_after_dock = (
            (pl.col('time_port_exit') - pl.col('time_departure'))
            .dt.total_minutes()/60
        ),
        #total time in port waters in hrs
        hrs_in_port_waters =(
            (pl.col('time_port_exit') - pl.col('time_port_entry'))
            .dt.total_minutes()/60
        )
    )
    #collect
    .collect()
)

#inspect
display(calls_df.describe())
calls_df.head(5)


statistic,call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters
str,str,str,str,str,str,f64,f64,str,str,str,str,f64,f64,f64,f64,f64
"""count""","""154454""","""154454""","""154454""","""154454""","""154454""",154453.0,154454.0,"""154454""","""154454""","""154454""","""154454""",154454.0,154454.0,154454.0,154454.0,154454.0
"""null_count""","""0""","""0""","""0""","""0""","""0""",1.0,0.0,"""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,null,null,1.0161e7,208.871619,"""2021-06-02 22:49:08.344471""","""2021-06-04 10:28:10.009718""","""2021-06-11 00:11:54.391521""","""2021-06-12 23:11:43.803818""",67.329997,14.661846,35.642629,46.989393,240.368338
"""std""",null,null,null,null,null,2.6897e7,59.443951,null,null,null,null,525.075158,175.955744,859.09431,1193.457777,2447.077442
"""min""","""205042000_Port of Long Beach, …","""Albany Port District, NY""","""ADM Corpus Christi Grain Eleva…","""00XE""","""205042000""",0.0,101.0,"""2018-01-01 00:35:19""","""2018-01-01 00:35:19""","""2018-01-01 07:15:53""","""2018-01-01 10:05:31""",0.083333,0.0,0.0,0.0,0.083333
"""25%""",null,null,null,null,null,9.295842e6,177.0,"""2019-08-23 11:41:22""","""2019-08-25 09:06:02""","""2019-08-31 03:57:02""","""2019-09-02 01:32:03""",16.566667,0.0,2.616667,2.116667,25.866667
"""50%""",null,null,null,null,null,9.401063e6,190.0,"""2021-07-13 22:30:32""","""2021-07-15 12:03:39""","""2021-07-23 06:09:31""","""2021-07-25 13:43:43""",31.216667,0.0,3.516667,2.783333,44.3
"""75%""",null,null,null,null,null,9.612143e6,232.0,"""2023-02-21 09:06:36""","""2023-02-23 00:27:48""","""2023-03-02 12:27:04""","""2023-03-04 18:38:22""",58.283333,0.0,5.433333,3.55,84.133333
"""max""","""725019920_Port Everglades, FL_…","""Wilmington, NC""","""YUSEN TERMINALS BERTHS 212-221""","""1JHK""","""725019920""",9.901843e8,667.0,"""2024-09-30 12:54:53""","""2024-09-30 16:32:15""","""2024-09-30 23:22:34""","""2024-09-30 23:22:34""",59292.65,52600.6,59117.283333,58950.283333,59127.133333


call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters
str,str,str,str,str,f64,f64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],f64,f64,f64,f64,f64
"""305134000_Port Everglades, FL_…","""Port Everglades, FL""","""PORT EVERGLADES BERTHS 14 & 15""","""0YXV""","""305134000""",9.435818e6,139.0,2020-07-01 07:52:29,2020-07-01 12:17:30,2020-07-03 07:05:00,2020-07-03 10:05:52,42.783333,0.0,4.416667,3.0,50.216667
"""305979000_Portland, ME_2024-05…","""Portland, ME""","""PORTLAND INTERNATIONAL MARINE …","""0YH1""","""305979000""",9.429194e6,158.0,2024-05-28 09:07:40,2024-05-28 11:19:36,2024-05-29 17:22:04,2024-05-29 19:25:43,30.033333,0.0,2.183333,2.05,34.3
"""366563000_Tacoma, WA_2022-09-1…","""Tacoma, WA""","""PORT OF TACOMA, HUSKY TERMINAL…","""0UFH""","""366563000""",7.907996e6,262.0,2022-09-10 17:11:48,2022-09-10 18:28:49,2022-09-10 21:47:57,2022-09-11 00:09:18,3.316667,0.0,1.283333,2.35,6.95
"""311297000_San Juan, PR_2019-07…","""San Juan, PR""","""SHED G""","""0N5V""","""311297000""",9.046502e6,150.0,2019-07-18 10:35:23,2019-07-18 13:12:24,2019-07-18 18:47:56,2019-07-18 21:07:53,5.583333,0.0,2.616667,2.316667,10.533333
"""255806068_Port Everglades, FL_…","""Port Everglades, FL""","""PORT EVERGLADES BERTHS 30, 31 …","""0YSY""","""255806068""",9.395111e6,166.0,2018-08-19 22:52:50,2018-08-20 01:32:11,2018-08-20 19:21:50,2018-08-20 22:56:08,17.516667,0.0,2.65,3.566667,24.05


In [197]:
(calls_df.filter(pl.col('hrs_to_dock')==0).shape[0] /
calls_df.shape[0])

0.024505678065961385

In [198]:
((calls_df.filter((pl.col('hrs_to_dock')==0)&
                (pl.col('hrs_in_port_after_dock')==0)).shape[0]) /
calls_df.filter(pl.col('hrs_to_dock')==0).shape[0])

0.14002642007926025

### Notes on Calls Frame and additional cleaning

- hrs_in_port_after_dock can be 0 when a vessel docks in overlapping port waters and visits both ports. 
    - In such cases, hrs_in_port_waters will be ~ 1/2 that of vessels who only visit 1 port. 
- hrs_to_dock == 0 implies that the first time the vessel sent an AIS message while in port waters was while at dock. This would be expected with new or reset AIS transcievers, and account for ~2.5% (3785) of port calls. 
    - Of these, ~14% (530) were docked in overlapping port waters prior to visiting the next dock. This would result in hrs_in_port_waters == hrs_at_berth. 

The mean and quartile statistics for the port calls seems reasonable; however, some calls have very long (6+ years in some cases) hrs_at_berth and related stats. This would result from vessel AIS transponders going offline at some stage during their visit to port waters. Rectifying this issue will be done either in the AIS ingestion or geodata_prep stages at a later date. For now we simply drop these as outliers. 

In [199]:
#define zero values drop
def drop_zero_values(df, cols):
    '''
    Drops zero values from the dataframe for the specified columns.
    Args:
        df: Polars DataFrame
        cols: List of columns to drop zero values from
    Returns:
        Polars DataFrame with zero values dropped
    '''
    for col in cols:
        df = df.filter(pl.col(col) > 0)
    return df

#define outlier drop
def drop_outliers(df, cols, threshold=3):
    '''
    Drops outliers from the dataframe for the specified columns.
    Args:
        df: Polars DataFrame
        cols: List of columns to drop outliers from
        threshold: Z-score threshold for outlier detection
    Returns:
        Polars DataFrame with outliers dropped
    '''
    print(f'Outlier threshold: {threshold} Std Devs')
    for col in cols:
        #compute z scores
        df = df.with_columns(
            z_score = (pl.col(col) - pl.col(col).mean()) / pl.col(col).std()
        )
        #drop outliers
        df = df.filter(pl.col('z_score').abs() < threshold)
    return df.drop('z_score')

In [200]:
#get count of rows from calls_df before drop
rows_prior = calls_df.shape[0]

#list cols that should never be zero by construction
zero_cols = ['hrs_at_berth', 'hrs_in_port_waters']
#list cols for outlier drop
outlier_cols = ['hrs_at_berth', 'hrs_to_dock', 'hrs_at_anchor', 
                'hrs_in_port_after_dock', 'hrs_in_port_waters']

#drop zero values
calls_df = drop_zero_values(calls_df, zero_cols)
#print number of observations dropped
print(f'Total zero rows dropped: {rows_prior - calls_df.shape[0]} of {rows_prior}')
rows_prior = calls_df.shape[0]

#drop outliers
#print z_score thresholds
for col in outlier_cols:
    print(f'{col} outlier threshold: {calls_df[col].std()*3/24:.1f} days')
#drop outliers
calls_df = drop_outliers(calls_df, outlier_cols, threshold=3)
#print rows dropped
print(f'Total outlier rows dropped: {rows_prior - calls_df.shape[0]} of {rows_prior}')

#inspect
display(calls_df.describe())
calls_df.head()

Total zero rows dropped: 0 of 154454
hrs_at_berth outlier threshold: 65.6 days
hrs_to_dock outlier threshold: 107.4 days
hrs_at_anchor outlier threshold: 22.0 days
hrs_in_port_after_dock outlier threshold: 149.2 days
hrs_in_port_waters outlier threshold: 305.9 days
Outlier threshold: 3 Std Devs
Total outlier rows dropped: 1437 of 154454


statistic,call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters
str,str,str,str,str,str,f64,f64,str,str,str,str,f64,f64,f64,f64,f64
"""count""","""153017""","""153017""","""153017""","""153017""","""153017""",153016.0,153017.0,"""153017""","""153017""","""153017""","""153017""",153017.0,153017.0,153017.0,153017.0,153017.0
"""null_count""","""0""","""0""","""0""","""0""","""0""",1.0,0.0,"""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,null,null,1.0150e7,208.758426,"""2021-06-05 08:51:40.650529""","""2021-06-05 22:20:39.958122""","""2021-06-08 05:36:19.480051""","""2021-06-08 11:37:06.153447""",49.051392,11.212385,13.475293,6.005277,74.748907
"""std""",null,null,null,null,null,2.6687e7,59.420839,null,null,null,null,71.673949,42.935658,43.545145,49.271169,119.634354
"""min""","""205042000_Port of Long Beach, …","""Albany Port District, NY""","""ADM Corpus Christi Grain Eleva…","""00XE""","""205042000""",0.0,101.0,"""2018-01-01 00:35:19""","""2018-01-01 00:35:19""","""2018-01-01 07:15:53""","""2018-01-01 10:05:31""",0.083333,0.0,0.0,0.0,0.083333
"""25%""",null,null,null,null,null,9.295945e6,176.0,"""2019-08-26 10:07:04""","""2019-08-26 20:44:57""","""2019-08-28 15:37:54""","""2019-08-28 22:05:30""",16.483333,0.0,2.633333,2.116667,25.733333
"""50%""",null,null,null,null,null,9.400928e6,190.0,"""2021-07-16 01:35:06""","""2021-07-16 18:12:36""","""2021-07-19 12:12:12""","""2021-07-19 16:51:58""",30.95,0.0,3.516667,2.783333,43.833333
"""75%""",null,null,null,null,null,9.611577e6,232.0,"""2023-02-23 20:41:20""","""2023-02-24 10:42:49""","""2023-02-26 17:34:59""","""2023-02-26 22:21:00""",57.566667,0.0,5.4,3.533333,82.216667
"""max""","""725019920_Port Everglades, FL_…","""Wilmington, NC""","""YUSEN TERMINALS BERTHS 212-221""","""1JHK""","""725019920""",9.849033e8,667.0,"""2024-09-30 12:54:53""","""2024-09-30 16:32:15""","""2024-09-30 21:54:12""","""2024-09-30 22:57:53""",1635.516667,522.6,2427.683333,3407.333333,4842.083333


call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters
str,str,str,str,str,f64,f64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],f64,f64,f64,f64,f64
"""305134000_Port Everglades, FL_…","""Port Everglades, FL""","""PORT EVERGLADES BERTHS 14 & 15""","""0YXV""","""305134000""",9.435818e6,139.0,2020-07-01 07:52:29,2020-07-01 12:17:30,2020-07-03 07:05:00,2020-07-03 10:05:52,42.783333,0.0,4.416667,3.0,50.216667
"""305979000_Portland, ME_2024-05…","""Portland, ME""","""PORTLAND INTERNATIONAL MARINE …","""0YH1""","""305979000""",9.429194e6,158.0,2024-05-28 09:07:40,2024-05-28 11:19:36,2024-05-29 17:22:04,2024-05-29 19:25:43,30.033333,0.0,2.183333,2.05,34.3
"""366563000_Tacoma, WA_2022-09-1…","""Tacoma, WA""","""PORT OF TACOMA, HUSKY TERMINAL…","""0UFH""","""366563000""",7.907996e6,262.0,2022-09-10 17:11:48,2022-09-10 18:28:49,2022-09-10 21:47:57,2022-09-11 00:09:18,3.316667,0.0,1.283333,2.35,6.95
"""311297000_San Juan, PR_2019-07…","""San Juan, PR""","""SHED G""","""0N5V""","""311297000""",9.046502e6,150.0,2019-07-18 10:35:23,2019-07-18 13:12:24,2019-07-18 18:47:56,2019-07-18 21:07:53,5.583333,0.0,2.616667,2.316667,10.533333
"""255806068_Port Everglades, FL_…","""Port Everglades, FL""","""PORT EVERGLADES BERTHS 30, 31 …","""0YSY""","""255806068""",9.395111e6,166.0,2018-08-19 22:52:50,2018-08-20 01:32:11,2018-08-20 19:21:50,2018-08-20 22:56:08,17.516667,0.0,2.65,3.566667,24.05


## Simple delay calculations

Differentiating between delay time and the "efficient" time it takes for a ship to get to a dock is somewhat difficult. 

At its most basic, we can calculate the difference between the hrs_to_dock time for each port call and the minimum hrs_to_dock for that vessel and dock. 

In [120]:
#add min hrs to dock for each vessel-dock pair
delay_df = (
    calls_df
    #min hrs to dock for each vessel-dock pair
    .with_columns(
        min_hrs_to_dock = pl.col('hrs_to_dock').min().over('mmsi', 'dock_id')
    )
    #"delay" in hrs
    .with_columns(
        hrs_delay = pl.col('hrs_to_dock') - pl.col('min_hrs_to_dock')
    )
    #drop unnecessary columns
    .drop('min_hrs_to_dock')
)

delay_df.describe()

statistic,call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters,hrs_delay
str,str,str,str,str,str,f64,f64,str,str,str,str,f64,f64,f64,f64,f64,f64
"""count""","""144130""","""144130""","""144130""","""144130""","""144130""",144129.0,144130.0,"""144130""","""144130""","""144130""","""144130""",144130.0,144130.0,144130.0,144130.0,144130.0,144130.0
"""null_count""","""0""","""0""","""0""","""0""","""0""",1.0,0.0,"""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,null,null,1.0156e7,208.435961,"""2021-05-28 12:27:41.285589""","""2021-05-29 01:56:17.903621""","""2021-05-31 05:21:18.349788""","""2021-05-31 10:53:45.817546""",46.134222,11.270176,13.468803,5.532976,70.426417,5.129696
"""std""",null,null,null,null,null,2.6814e7,59.22973,null,null,null,null,52.742429,42.969633,40.759136,29.204737,84.469003,26.008882
"""min""","""205042000_Port of Long Beach, …","""Albany Port District, NY""","""ADM Corpus Christi Grain Eleva…","""00XE""","""205042000""",0.0,101.0,"""2018-01-01 00:53:19""","""2018-01-01 01:51:47""","""2018-01-01 07:15:53""","""2018-01-01 10:05:31""",0.083333,0.0,0.083333,0.083333,1.966667,0.0
"""25%""",null,null,null,null,null,9.295397e6,176.0,"""2019-08-15 07:48:37""","""2019-08-15 23:45:52""","""2019-08-17 18:19:12""","""2019-08-17 23:10:37""",16.466667,0.0,2.7,2.2,25.783333,0.0
"""50%""",null,null,null,null,null,9.399777e6,190.0,"""2021-06-28 15:54:08""","""2021-06-29 08:15:15""","""2021-07-01 12:25:09""","""2021-07-01 16:52:12""",30.483333,0.0,3.566667,2.816667,43.416667,0.183333
"""75%""",null,null,null,null,null,9.610406e6,232.0,"""2023-02-19 16:11:48""","""2023-02-20 05:19:38""","""2023-02-22 11:49:33""","""2023-02-22 18:11:30""",55.95,0.0,5.483333,3.566667,80.55,1.25
"""max""","""725019920_Port Everglades, FL_…","""Wilmington, NC""","""YUSEN TERMINALS BERTHS 212-221""","""1JHK""","""725019920""",9.800025e8,667.0,"""2024-09-30 07:46:57""","""2024-09-30 11:36:06""","""2024-09-30 21:54:12""","""2024-09-30 22:57:53""",899.15,530.75,1685.45,1762.916667,1809.016667,1246.65


In [121]:
#count of calls with zero delay
delay_df.filter(pl.col('hrs_delay') == 0).shape[0]

62770

64k of 145k port calls align show zero delay indicating those calls represent the only time that that vessel visited that dock. 

### Time Awaiting Berth

We define time awaiting berth as the total time it takes a vessel to get to the dock minus the amount of time the dock was occupied while that vessel was en route. 

In [122]:
%%script echo skipping
#calculate time awaiting berth

#for each call_id and dock, get the total time dock was occupied between time_port_entry and time_arrival

#get time_port_entry, time_arrival for each call id
lf = (calls_df.select('call_id', 'time_port_entry', 'time_arrival')
      .unique().lazy())
#join to main lf
main_lf = main_lf.join(lf, on='call_id', how='left')

for call in calls_df.select('call_id').unique().to_series():
      #get start time and end time
      start = (calls_df.filter(pl.col('call_id')==call)
               .select('time_port_entry').item())
      end = (calls_df.filter(pl.col('call_id')==call)
               .select('time_arrival').item())
      #get dock occupancy
      df = (
            main_lf
            .with_columns(
                  dock_occupied = (
                        (pl.col('status')==5)
                        .then(pl.col('status_duration'))
                        .otherwise(pl.lit(None))
                  )
            )
      )


skipping


In [123]:
#create monthly stats dataframe
monththly_df = (
    calls_df
    #get month from docking time
    .with_columns(
        #extract month from docking time
        month = pl.col('time_arrival').dt.strftime('%Y%m')
    )
    #group by port dock and month
    .group_by(['port_name', 'dock_id', 'month'])
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('call_id'),
        #time at dock stats for each vessel in hours
        hrs_occupied = pl.sum('hrs_at_berth'),
        hrs_at_berth_median = pl.median('hrs_at_berth'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth'),
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor'),
        #time in port waters 
        hrs_in_port_waters_total = pl.sum('hrs_in_port_waters'),
        hrs_in_port_waters_mean = pl.mean('hrs_in_port_waters'),
        hrs_in_port_waters_median = pl.median('hrs_in_port_waters')
    )
    #get hours from each month
    .with_columns(
        hrs_in_month = (
            pl.when(pl.col('month').str.tail(2).is_in(['01', '03', '05', '07',
                                                       '08', '10', '12']))
            .then(31*24)
            .when(pl.col('month').str.tail(2).is_in(['04', '06', '09', '11']))
            .then(30*24)
            .otherwise(28*24)
        )
    )
    .with_columns(
        #dock utilization - percentage of time a dock is occupied
        utilization = (
            pl.col('hrs_occupied')/pl.col('hrs_in_month')
        )
    )
    #drop hours in month
    .drop('hrs_in_month')
    #sort by port dock then month
    .sort(['port_name', 'dock_id', 'month'])
)

#inspect
display(monththly_df.describe())
monththly_df.head()

statistic,port_name,dock_id,month,vessels,vessel_size_mean,calls,hrs_occupied,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean,hrs_in_port_waters_total,hrs_in_port_waters_mean,hrs_in_port_waters_median,utilization
str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""34058""","""34058""","""34058""",34058.0,34058.0,34058.0,34058.0,34058.0,34058.0,34058.0,34058.0,34058.0,34058.0,34058.0,34058.0
"""null_count""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,3.726232,196.352468,4.231899,195.235346,57.711677,60.308419,9.426398,12.243099,298.03745,87.476779,82.498843,0.26762
"""std""",null,null,null,3.869374,49.397631,4.657953,183.703496,59.203678,59.118048,33.670132,35.344189,328.894733,83.192082,82.518792,0.251695
"""min""","""Albany Port District, NY""","""00XE""","""201801""",1.0,101.0,1.0,0.15,0.15,0.15,0.0,0.0,3.0,3.0,3.0,0.000202
"""25%""",null,null,null,1.0,168.5,1.0,65.016667,22.766667,25.04,0.0,0.0,94.916667,37.275,34.016667,0.088978
"""50%""",null,null,null,3.0,185.0,3.0,141.966667,40.233333,43.25,0.0,0.0,200.45,63.05,57.566667,0.194792
"""75%""",null,null,null,5.0,212.333333,5.0,267.533333,71.55,75.21,0.0,5.044444,381.183333,107.936667,101.066667,0.366846
"""max""","""Wilmington, NC""","""1JHK""","""202409""",49.0,379.6,72.0,2357.966667,873.883333,873.883333,516.566667,516.566667,6477.016667,1689.45,1689.45,3.16931


port_name,dock_id,month,vessels,vessel_size_mean,calls,hrs_occupied,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean,hrs_in_port_waters_total,hrs_in_port_waters_mean,hrs_in_port_waters_median,utilization
str,str,str,u32,f64,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albany Port District, NY""","""0PST""","""201803""",1,171.0,1,67.666667,67.666667,67.666667,0.0,0.0,73.283333,73.283333,73.283333,0.09095
"""Albany Port District, NY""","""0PST""","""201804""",1,189.0,1,165.6,165.6,165.6,0.0,0.0,172.833333,172.833333,172.833333,0.23
"""Albany Port District, NY""","""0PST""","""201806""",1,179.0,1,118.4,118.4,118.4,0.0,0.0,125.083333,125.083333,125.083333,0.164444
"""Albany Port District, NY""","""0PST""","""201807""",2,199.0,2,166.416667,83.208333,83.208333,0.0,0.0,200.683333,100.341667,100.341667,0.223678
"""Albany Port District, NY""","""0PST""","""201808""",3,192.666667,3,185.0,78.45,61.666667,0.0,0.0,350.9,116.966667,98.483333,0.248656


#### Hours calc discussion

The current code first associates the call_id with the month in which the vessel arrived at dock, then counts total times for that call_id to that month. This results in some edge cases where hours stats far exceed the total hours in the month, as in the case that a vessel arrives at the dock and stays there for a very long period of time. 

This can be partially resolved by dropping statuses that are very long, which needs to be done anyway.
- what's the right strategy? set status duration to (the median for that dock? zero? 12hr?) and give an unknown status afterwards? 

It would be fully resolved by totaling monthly hrs (at dock or hrs utilized, for example) independently of call_id.  

In [124]:
#create annual dock stats dataframe
annual_docks_df = (
    calls_df
    #get year from docking time
    .with_columns(
        #extract year from docking time
        year = pl.col('time_arrival').dt.year()
    )
    #group by port dock and month
    .group_by(['port_name', 'dock_id', 'year'])
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('call_id'),
        #time at dock stats for each vessel in hours
        hrs_at_berth_median = pl.median('hrs_at_berth'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth'),
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor')
    )
    #sort by port then month
    .sort(['port_name', 'year'])
)

#inspect
annual_docks_df.head()

port_name,dock_id,year,vessels,vessel_size_mean,calls,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean
str,str,i32,u32,f64,u32,f64,f64,f64,f64
"""Albany Port District, NY""","""0Y78""",2018,2,158.0,2,39.808333,39.808333,0.0,0.0
"""Albany Port District, NY""","""0Q86""",2018,2,141.0,2,38.833333,38.833333,0.0,0.0
"""Albany Port District, NY""","""0Y0T""",2018,3,131.25,4,63.25,67.55,0.0,0.0
"""Albany Port District, NY""","""0RQB""",2018,4,140.0,4,27.591667,27.079167,0.0,0.0
"""Albany Port District, NY""","""0PST""",2018,9,189.333333,9,78.45,85.548148,0.0,0.0


In [125]:
#create annual port stats dataframe
annual_ports_df = (
    calls_df
    #get year from docking time
    .with_columns(
        #extract year from docking time
        year = pl.col('time_arrival').dt.year()
    )
    #group by port dock and month
    .group_by(['port_name', 'year'])
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('call_id'),
        #time at dock stats for each vessel in hours
        hrs_at_berth_median = pl.median('hrs_at_berth'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth'),
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor')
    )
    #sort by port then month
    .sort(['port_name', 'year'])
)

#inspect
annual_ports_df.head()

port_name,year,vessels,vessel_size_mean,calls,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean
str,i32,u32,f64,u32,f64,f64,f64,f64
"""Albany Port District, NY""",2018,56,153.758065,62,66.091667,68.102419,0.0,0.008871
"""Albany Port District, NY""",2019,29,156.15625,32,68.65,74.980208,0.0,0.01875
"""Albany Port District, NY""",2020,32,147.314286,35,67.55,78.772857,0.0,0.0
"""Albany Port District, NY""",2021,44,156.869565,46,63.183333,74.366304,0.0,0.0
"""Albany Port District, NY""",2022,54,159.689655,58,71.666667,82.093391,0.0,1.559483


In [126]:
ports_alltime_df = (
    calls_df
    #group by port 
    .group_by('port_name')
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('call_id'),
        #time at dock stats for each vessel in hours
        hrs_at_berth_median = pl.median('hrs_at_berth'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth'),
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor')
    )
    #sort by port
    .sort('port_name')
)
#inspect
display(ports_alltime_df.describe())
ports_alltime_df.head()

statistic,port_name,vessels,vessel_size_mean,calls,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean
str,str,f64,f64,f64,f64,f64,f64,f64
"""count""","""70""",70.0,70.0,70.0,70.0,70.0,70.0,70.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,687.042857,196.56326,2059.0,43.327262,57.852461,0.9625,10.290437
"""std""",null,695.715947,36.591207,2236.442581,43.115369,53.114338,3.168368,12.320886
"""min""","""Albany Port District, NY""",3.0,116.550459,3.0,8.141667,10.490179,0.0,0.0
"""25%""",null,129.0,172.580707,384.0,22.316667,32.077986,0.0,1.260494
"""50%""",null,518.0,189.856935,1277.0,33.7,47.307092,0.0,5.364461
"""75%""",null,971.0,215.851181,3117.0,48.091667,62.525686,0.0,15.424482
"""max""","""Wilmington, NC""",2663.0,296.610925,10316.0,293.441667,357.611111,18.783333,52.985893


port_name,vessels,vessel_size_mean,calls,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean
str,u32,f64,u32,f64,f64,f64,f64
"""Albany Port District, NY""",248,157.056657,353,64.083333,75.395751,0.0,0.261473
"""Anacortes, WA""",249,219.277778,1062,36.95,48.917875,0.0,31.772316
"""Baltimore, MD""",2509,214.193535,7394,33.25,45.41407,0.0,15.424482
"""Beaumont, TX""",971,183.041391,1812,53.708333,64.827281,0.0,4.176702
"""Boston, MA""",293,296.610925,897,17.65,24.456652,0.0,2.436009


## Visualizations

In [127]:
#make geodataframe
#get port location data
df = main_lf.select('port_name', 'port_lat', 'port_lon').unique().collect()
#convert to geopandas
gdf = gpd.GeoDataFrame(
    ports_alltime_df
    #join to port location data
    .join(df, on='port_name', how='left')
    #convert to pandas dataframe
    .to_pandas(),
    #set geometry
    geometry = gpd.points_from_xy(df['port_lon'], df['port_lat'], crs=4326)
)
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   port_name             70 non-null     object  
 1   vessels               70 non-null     uint32  
 2   vessel_size_mean      70 non-null     float64 
 3   calls                 70 non-null     uint32  
 4   hrs_at_berth_median   70 non-null     float64 
 5   hrs_at_berth_mean     70 non-null     float64 
 6   hrs_at_anchor_median  70 non-null     float64 
 7   hrs_at_anchor_mean    70 non-null     float64 
 8   port_lat              70 non-null     float64 
 9   port_lon              70 non-null     float64 
 10  geometry              70 non-null     geometry
dtypes: float64(7), geometry(1), object(1), uint32(2)
memory usage: 5.6+ KB


In [128]:
#scatterplot
fig = px.scatter_geo(
    gdf,
    lon='port_lon',
    lat='port_lat',
    size='vessels',
    color='hrs_at_berth_median',
    range_color=[0,50],
    hover_name='port_name',
    size_max=30,
    title='Total Vessels (all time) and Median Hours at Berth',
    color_continuous_scale=px.colors.sequential.Viridis,
    width=1000,
    height=600,
    labels={
        'time_at_berth_avg':'Hours at Berth'
    }
)

# Fit the view to ports
fig.update_geos(fitbounds="locations")

# Add footnote using add_annotation
fig.add_annotation(
    text="Note: Circle size corresponds to averages vessels per month",  # Footnote text
    xref="paper", yref="paper",  # Position relative to the plot area
    x=0, y=0-0.05,  # Adjust to footnote position
    showarrow=False,  # No arrow, just text
    font=dict(size=14, color="black"),  # Customize the font style
    align="left"
)

# Show the figure
fig.show()

## Point in Time Stats

Still under development

In [129]:
#get point in time stats

#create point in time (pit) df to join stats to
pit_df = (
    main_lf
    .with_columns(
        date = pl.col('time').dt.date(),
        month = pl.col('time').dt.strftime('%Y%m')
    )
    .select('port_name', 'dock_id', 'month', 'date')
    .unique().collect()
)

for hour in range(0, 24):
    #create a time object for each hour
    hour_dt = pl.time(hour)
    #create a dataframe for each hour
    hour_df = (
        main_lf
        .with_columns(
            #get end of status time
            end_time = pl.col('time') + pl.col('status_duration'),
            #get date from time
            date = pl.col('time').dt.date(),
            #get month from time
            month = pl.col('time').dt.strftime('%Y%m')
        )
        #group by port dock and hour
        .group_by(['port_name', 'dock_id', 'month', 'date'])
        .agg(
            #number of vessels at dock at each hour
            vessels_at_dock = (
                #when moored at hour
                pl.when((pl.col('status')==5) & 
                        (hour_dt.is_between(pl.col('time').dt.time(), 
                                         pl.col('end_time').dt.time())))
                #then count the individual vessels
                .then(pl.col('mmsi'))
                .otherwise(pl.lit(None))
                .drop_nulls() #n_unique counts nulls as unique values
                .n_unique()
            ),
            #number of vessels at anchor at each hour
            vessels_at_anchor = (
                #when anchored at hour
                pl.when((pl.col('status')==1) & 
                        (hour_dt.is_between(pl.col('time').dt.time(), 
                                        pl.col('end_time').dt.time())))
                #then count the individual vessels
                .then(pl.col('mmsi'))
                .otherwise(pl.lit(None))
                .drop_nulls()
                .n_unique()
            )
        )
        .collect()
    )
    #join the hour dataframe to the main pit dataframe
    pit_df = (
        pit_df
        .join(hour_df, 
              on=['port_name', 'dock_id', 'month', 'date'], 
              how='left')
        #rename the columns to include the hour
        .rename({
            'vessels_at_dock': f'vessels_at_dock_{hour}',
            'vessels_at_anchor': f'vessels_at_anchor_{hour}'
        })
    )

#get port stats by month
pit_df = (
    pit_df
    #group by port and date
    .group_by(['port_name', 'month', 'date'])
    .agg(
        #sum the number of vessels at all docks at each hour
        cs.starts_with('vessels_at_dock_').sum(),
        #sum the number of vessels at anchor at each hour
        cs.starts_with('vessels_at_anchor_').sum()
    )
    #get the max at any hour
    .with_columns(
        #get max at dock at any hour
        vessels_at_dock_max = (
            pl.max_horizontal(cs.starts_with('vessels_at_dock_'))
        ),
        #get mean at dock any hour
        vessels_at_dock_mean = (
            pl.mean_horizontal(cs.starts_with('vessels_at_dock_'))
        ),
        #get max at anchor at any hour
        vessels_at_anchor_max = (
            pl.max_horizontal(cs.starts_with('vessels_at_anchor_'))
        ),
        #get mean at anchor any hour
        vessels_at_anchor_mean = (
            pl.mean_horizontal(cs.starts_with('vessels_at_anchor_'))
        )
    )
    #select the columns to keep
    .select(['port_name', 'month', 'date', 'vessels_at_dock_max', 
             'vessels_at_dock_mean', 'vessels_at_anchor_max',
             'vessels_at_anchor_mean'])
    #aggregate by month
    .group_by(['port_name', 'month'])
    .agg(
        #get max at anchor on any date during that month
        vessels_at_anchor_max = pl.max('vessels_at_anchor_max'),
        #get mean at anchor on any date during that month
        vessels_at_anchor_mean = pl.mean('vessels_at_anchor_mean'),
        #get max at dock on any date during that month
        vessels_at_dock_max = pl.max('vessels_at_dock_max'),
        #get mean at dock on any date during that month
        vessels_at_dock_mean = pl.mean('vessels_at_dock_mean'
        )
    )
)

In [130]:
#inspect
display(pit_df.describe())
pit_df.head()

statistic,port_name,month,vessels_at_anchor_max,vessels_at_anchor_mean,vessels_at_dock_max,vessels_at_dock_mean
str,str,str,f64,f64,f64,f64
"""count""","""5201""","""5201""",5201.0,5201.0,5201.0,5201.0
"""null_count""","""0""","""0""",0.0,0.0,0.0,0.0
"""mean""",null,null,0.793309,0.048114,2.650067,0.508413
"""std""",null,null,0.814863,0.09063,1.826468,0.497506
"""min""","""Albany Port District, NY""","""201801""",0.0,0.0,0.0,0.0
"""25%""",null,null,0.0,0.0,1.0,0.195076
"""50%""",null,null,1.0,0.014493,2.0,0.358025
"""75%""",null,null,1.0,0.05787,3.0,0.622024
"""max""","""Wilmington, NC""","""202409""",6.0,0.995833,13.0,3.938889


port_name,month,vessels_at_anchor_max,vessels_at_anchor_mean,vessels_at_dock_max,vessels_at_dock_mean
str,str,u32,f64,u32,f64
"""Portland, ME""","""202312""",1,0.025641,3,0.291667
"""Calhoun Port Authority, TX""","""202205""",3,0.144231,3,0.238782
"""Anacortes, WA""","""202108""",1,0.043478,2,0.26087
"""Port of Greater Baton Rouge, L…","""202003""",2,0.065104,1,0.096354
"""Kalaeloa Barbers Point, HI""","""202011""",0,0.0,1,0.260417


### Max/Mean stats for vessels_at_dock 

- Current output seems far too low - e.g. Port of LA shows a max of 11 vessels at dock at any time since 2018; since there are 37 docks at LA we expect a max in the 20s or higher. 

In [131]:
pit_df.filter(pl.col('port_name')=='Port of Los Angeles, CA').sort('month')

port_name,month,vessels_at_anchor_max,vessels_at_anchor_mean,vessels_at_dock_max,vessels_at_dock_mean
str,str,u32,f64,u32,f64
"""Port of Los Angeles, CA""","""201801""",4,0.319892,7,1.858871
"""Port of Los Angeles, CA""","""201802""",3,0.27381,5,1.160714
"""Port of Los Angeles, CA""","""201803""",2,0.260753,9,1.829301
"""Port of Los Angeles, CA""","""201804""",3,0.35,7,1.573611
"""Port of Los Angeles, CA""","""201805""",2,0.235215,5,1.444892
"""Port of Los Angeles, CA""","""201806""",2,0.247222,6,1.544444
"""Port of Los Angeles, CA""","""201807""",3,0.319892,6,1.495968
"""Port of Los Angeles, CA""","""201808""",5,0.262097,7,1.717742
"""Port of Los Angeles, CA""","""201809""",3,0.3,7,1.483333
